# Libraries

In [2]:
import os
from pathlib import Path
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import pandas as pd
#For registration
import voxelmorph as vxm
import tensorflow as tf
assert tf.__version__.startswith('2.'), 'This tutorial assumes Tensorflow 2.0+'
#Own libraries
from metadata import ImageDataset, patient
import utils

2023-01-07 16:55:06.207224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 16:55:06.439613: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-07 16:55:06.474148: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-07 16:55:06.474162: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
notebooks_path = Path.cwd()
repo_path = notebooks_path.parent
print(f'The current directory is: {notebooks_path}')

The current directory is: /home/ricardino/Documents/MAIA/tercer_semestre/MISA/final_project/MISA_FINAL_PROJECT/notebooks


# MAIN

We want to register each training image to the validation images, individually.<br>

Saving the registered images may help us save some time in the future, as we will use them for other applications.
The question would be to decide which parameter file to use for the registration.
Thankfully, there exists Voxelmorph which is a deep learning pretrained model that can be used for registration.

## Voxelmorph

In [1]:
def voxmorph_register(fixed_path:str, moving_path:str, label_path:str):
    """register moving image to fixed image using voxmorph, also register label image

    Args:
        fixed_path (str): fixed image path
        moving_path (str): moving image path
        label_path (str): moving image label path

    Returns:
        4 objs: moved image, moved label, warp, fixed image
    """

    #load model
    model_path = str(repo_path / 'data/voxelmorph/vxm_dense_brain_T1_3D_mse.h5')

    # tensorflow device handling
    device, nb_devices = vxm.tf.utils.setup_device()

    # load moving, labels and fixed images
    add_feat_axis = not False
    moving = vxm.py.utils.load_volfile(moving_path, add_batch_axis=True, add_feat_axis=add_feat_axis)
    label = vxm.py.utils.load_volfile(label_path, add_batch_axis=True, add_feat_axis=add_feat_axis)
    #fill all background values to GM, value 2
    label[label == 0] = 2
    fixed, fixed_affine = vxm.py.utils.load_volfile( #fixed afine is basically the dimenionallity of the image
        fixed_path, add_batch_axis=True, add_feat_axis=add_feat_axis, ret_affine=True)
    
    # shape
    inshape = moving.shape[1:-1]
    nb_feats = moving.shape[-1]

    with tf.device(device):
        # load configuration
        config = dict(inshape=inshape, input_model=None)
        #register and get warp
        warp = vxm.networks.VxmDense.load(model_path, **config).register(moving, fixed)
        #use warp on moving images and labels
        moved = vxm.networks.Transform(inshape, nb_feats=nb_feats).predict([moving, warp])
        moved_label = vxm.networks.Transform(inshape, nb_feats=nb_feats).predict([label, warp])
        
    #mask label using moved image
    moved_label[moved==0] = 0
    
    return moved, moved_label, warp, fixed_affine


def compute_maattesMI(moved_path:str, fixed_path:str):
    """Computes the Mattes mutual information between two images.

    Args:
        moved_path (str): path of moving image
        fixed_path (str): path of fixed image

    Returns:
        float: metric value
    """
    
    img1 = sitk.ReadImage(moved_path)
    img2 = sitk.ReadImage(fixed_path)
    #change dtype of moved image to float 64
    img1 = sitk.Cast(img1, sitk.sitkFloat64)

    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation()
    metric = registration_method.MetricEvaluate(img1,img2)
    
    return metric

In [ ]:
#define datasets
im_data_train = ImageDataset(set_name='Training')
im_data_val = ImageDataset(set_name='Validation')


for id_val in tqdm(im_data_val.IDs):
    #define dataframe to save results
    df_val = None
    for id_train in tqdm(im_data_train.IDs):
        #Moving image
        #id_train = im_data_train.IDs[5]
        pat_train = patient(id_train, im_data_train)
        moving_path = pat_train.im_path_norm
        label_path = pat_train.labels_path

        #Fixed image
        #id_val = im_data_val.IDs[0]
        pat_val = patient(id_val, im_data_val)
        fixed_path = pat_val.im_path_norm
        ground_truth = pat_val.labels(format='np')

        #register using voxelmorph
        moved, moved_label, warp, fixed_affine = voxmorph_register(fixed_path, moving_path, label_path)

        # save moved image and label
        moved_path = str(repo_path / f'data/voxelmorph/moved_{id_train}_to_{id_val}.nii.gz')
        moved_label_path = str(repo_path / f'data/voxelmorph/moved_labels_{id_train}_to_{id_val}.nii.gz')
        vxm.py.utils.save_volfile(moved.squeeze(), moved_path, fixed_affine)
        vxm.py.utils.save_volfile(np.rint(moved_label.squeeze()), moved_label_path, fixed_affine)

        #compute metric
        metric = compute_maattesMI(moved_path, fixed_path)
        #compute dice
        new_label = sitk.GetArrayFromImage(sitk.ReadImage(moved_label_path))
        dice = [utils.dice_score(new_label==tissue,ground_truth==tissue) for tissue in range(1,4)]
        #save in dataframe. Columns are dice CSF, dice GM, dice WM, metric
        df = pd.DataFrame([[id_train] + dice+[metric]], columns=['id_train', 'CSF', 'GM', 'WM', 'metric'])
        #concatenate with previous results
        df_val = pd.concat([df_val, df], ignore_index=True)
    #arange in descending order (most negative metric is the best)
    df_val = df_val.sort_values(by='metric', ascending=True)
    #save dataframe
    df_val.to_csv(str(repo_path / f'data/results/most_similar/most_similar_{id_val}.csv'), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:124: UserWarning: int_downsize is deprecated, use the int_resolution parameter.
  warnings.warn('int_downsize is deprecated, use the int_resolution parameter.')
/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:129: UserWarning: unet_half_res is deprecated, use the svf_resolution parameter.
  warnings.warn('unet_half_res is deprecated, use the svf_resolution parameter.')


1/1 [==============================] - 1s 773ms/step


/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:124: UserWarning: int_downsize is deprecated, use the int_resolution parameter.
  warnings.warn('int_downsize is deprecated, use the int_resolution parameter.')
/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:129: UserWarning: unet_half_res is deprecated, use the svf_resolution parameter.
  warnings.warn('unet_half_res is deprecated, use the svf_resolution parameter.')


1/1 [==============================] - 1s 816ms/step


/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:124: UserWarning: int_downsize is deprecated, use the int_resolution parameter.
  warnings.warn('int_downsize is deprecated, use the int_resolution parameter.')
/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:129: UserWarning: unet_half_res is deprecated, use the svf_resolution parameter.
  warnings.warn('unet_half_res is deprecated, use the svf_resolution parameter.')


...

/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:124: UserWarning: int_downsize is deprecated, use the int_resolution parameter.
  warnings.warn('int_downsize is deprecated, use the int_resolution parameter.')
/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:129: UserWarning: unet_half_res is deprecated, use the svf_resolution parameter.
  warnings.warn('unet_half_res is deprecated, use the svf_resolution parameter.')


1/1 [==============================] - 1s 762ms/step


/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:124: UserWarning: int_downsize is deprecated, use the int_resolution parameter.
  warnings.warn('int_downsize is deprecated, use the int_resolution parameter.')
/home/ricardino/anaconda3/envs/misa_final/lib/python3.9/site-packages/voxelmorph/tf/networks.py:129: UserWarning: unet_half_res is deprecated, use the svf_resolution parameter.
  warnings.warn('unet_half_res is deprecated, use the svf_resolution parameter.')


1/1 [==============================] - 1s 801ms/step


100%|██████████| 5/5 [07:53<00:00, 94.79s/it]


In [15]:
#Now we get the first line of each dataframe, which is the most similar label propagation
df_most_similar = None
for id_val in im_data_val.IDs:
    csv_path = str(repo_path / f'data/results/most_similar/most_similar_{id_val}.csv')
    #read csv
    df = pd.read_csv(csv_path, dtype={'id_train': str})
    #add id_val to dataframe at the beginning
    df.insert(0, 'id_val', id_val)
    #concatenate with previous results
    df_most_similar = pd.concat([df_most_similar, df.iloc[[0]]], ignore_index=True)
#save dataframe
df_most_similar.to_csv(str(repo_path / f'data/results/most_similar_dice.csv'), index=False)

Adding the other two main metrics, we have to check the results for the selected most similar atlas.

In [24]:
#open dataset validation
im_data_val = ImageDataset(set_name='Validation')

df_mostSimilar_metrics = None
#open the most similar dataframe
df_most_similar = pd.read_csv(str(repo_path / f'data/results/most_similar_dice.csv'), dtype={'id_val':str, 'id_train':str})
for id_val, id_train in zip(df_most_similar['id_val'], df_most_similar['id_train']):
    pat_val  = patient(id_val, im_data_val)
    #open segmentation
    seg_path = repo_path / 'data'/'voxelmorph'/f'moved_labels_{id_train}_to_{id_val}.nii.gz'
    seg = utils.getArrayfromPath(seg_path)
    #open ground truth
    gTruth = pat_val.labels(format='np')
    #compute the metrics
    df_metrics = utils.compute_metrics(seg, pat_val, id_val)
    #concatenate in df_bayesian
    df_mostSimilar_metrics = pd.concat([df_mostSimilar_metrics, df_metrics], axis=0)
#save as csv
df_mostSimilar_metrics.to_csv(str(repo_path / f'data/results/most_similar_metrics.csv'), index=False)